In [1]:
#!pip install neptune
#!pip install optuna
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from medmnist import DermaMNIST
import random
import neptune
import itertools
import optuna

In [2]:
torch.cuda.empty_cache()
class CNN(nn.Module):
    def __init__(self, dropout_rate):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

        self.flattened_size = self.get_flattened_size((3, 64, 64))
        self.fc1 = nn.Linear(self.flattened_size, 64)
        self.fc2 = nn.Linear(64, 7)

    def get_flattened_size(self, input_shape):
        x = torch.zeros(1, *input_shape)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Funkcja celu dla Optuny
def objective(trial):
    # Hiperparametry do tuningu
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)

    # Dane
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = DermaMNIST(root='./data', split="train", download=True, transform=transform, size=64)
    test_dataset = DermaMNIST(root='./data', split="test", download=True, transform=transform, size=64)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model
    model = CNN(dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Trening
    for epoch in range(10):  # Mała liczba epok do szybkiego tuningu
        model.train()
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            target = target.long()  # Ensure target is of type torch.long
            if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

    # Ewaluacja
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            target = target.long()  # Ensure target is of type torch.long
            if len(target.size()) > 1:
                target = target.squeeze()  # Remove extra dimensions
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

    accuracy = correct / total
    return 1 - accuracy  # Minimalizujemy 1 - accuracy

# Główna funkcja
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    study = optuna.create_study(direction="minimize")  # Minimalizacja (1 - accuracy)
    study.optimize(objective, n_trials=50)

    # Najlepsze hiperparametry
    print("Najlepsze hiperparametry: ", study.best_params)
    print("Najlepszy wynik: ", 1 - study.best_value)

[I 2025-01-24 11:57:50,961] A new study created in memory with name: no-name-d28e7835-6306-423e-9b66-d7467e368051
C:\Users\Kamil\AppData\Local\Temp\ipykernel_34776\4284873989.py:42: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
C:\Users\Kamil\AppData\Local\Temp\ipykernel_34776\4284873989.py:43: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform("dropout_rate", 0.2, 0.5)


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 11:58:10,005] Trial 0 finished with value: 0.26034912718204484 and parameters: {'batch_size': 32, 'learning_rate': 0.0028545696530108368, 'dropout_rate': 0.2946138903777211}. Best is trial 0 with value: 0.26034912718204484.


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 11:58:29,046] Trial 1 finished with value: 0.23341645885286788 and parameters: {'batch_size': 32, 'learning_rate': 0.00017184043477096354, 'dropout_rate': 0.238695650594483}. Best is trial 1 with value: 0.23341645885286788.


...

Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 12:17:05,001] Trial 48 finished with value: 0.2354114713216957 and parameters: {'batch_size': 128, 'learning_rate': 0.00044154845805712337, 'dropout_rate': 0.3206355646694109}. Best is trial 11 with value: 0.2194513715710723.


Using downloaded and verified file: ./data\dermamnist_64.npz
Using downloaded and verified file: ./data\dermamnist_64.npz


[I 2025-01-24 12:17:32,768] Trial 49 finished with value: 0.2673316708229426 and parameters: {'batch_size': 64, 'learning_rate': 0.0016104443684200702, 'dropout_rate': 0.29417416304071786}. Best is trial 11 with value: 0.2194513715710723.


Najlepsze hiperparametry:  {'batch_size': 32, 'learning_rate': 0.00010948895358182584, 'dropout_rate': 0.2872653938750536}
Najlepszy wynik:  0.7805486284289277
